# ALL FILES

In [ ]:
###########
# IMPORTS #
###########
from sentence_transformers import SentenceTransformer
import re
import pickle
import os

import sys
home_path=open('/home/ec2-user/SageMaker/mariano/repositories/refugee-research-problem-hrir/config/repository_path.txt').read()
sys.path.append(home_path)

from lib import tdmstudio
from lib import refugee_data_sources

####################
# INPUT AND OUTPUT #
####################
model_path='/home/ec2-user/SageMaker/mariano/sentence_bert/pretrained/all-MiniLM-L6-v2/'



files = [os.path.join(home_path,'embeddings', file_) for file_ in os.listdir(os.path.join(home_path,'embeddings/')) if re.match('item_representation_[0-9]*.pickle', file_)]
numbers = sorted([int(re.findall('item_representation_([0-9]*).pickle',file_)[0]) for file_ in files])

next_number=numbers[-1]+1

if len(numbers)>0:
    output = os.path.join(home_path, f'embeddings/item_representation_{next_number}.pickle')
    print(f'Found previous files: {numbers}')
    print(f'Creating file: {output}')
else:
    output = os.path.join(home_path, f'embeddings/item_representation_1.pickle')
    print('Creating item_representation, first file.')


already_process=set()

for file_ in files:
    with open(file_, 'rb') as reader:
        already_process = already_process.union(pickle.load(reader))
print(f'Previous files found. Already processed: {len(already_process)}')

    


In [3]:
print('Loading model                 ...')
model = SentenceTransformer(model_path)

print('Loading file names            ...')

cap=4000000
print(f'[WARNING] working with cap={cap}')

unlabeled = refugee_data_sources.get_unlabeled()[:cap]
print(f'File found:                  {len(unlabeled)}')
print(f'Filtering already processed: {len(already_process)}')
unlabeled = list(filter(lambda id_: not id_ in already_process,unlabeled))


print(f'Running over                  {len(unlabeled)} elements ...')
step=50000

for i in range(0,len(unlabeled), step):
    fin = min(i+step,len(unlabeled))
    print(f'{i}-{fin}')
    vecs = model.encode(list(map(tdmstudio.get_title_and_text, map(tdmstudio.get_filename, unlabeled[i:fin]))))
    item_representation =  {id_:vecs[ix,:] for ix,id_ in enumerate(unlabeled[i:fin])}
    print('Dumping to disk ...')
    print(output)
    with open(output,'wb') as writer:
        pickle.dump(item_representation, writer)
    
    next_number=next_number+1
    output = os.path.join(home_path, f'embeddings/item_representation_{next_number}.pickle')
    del vecs

print('Done ! ')

Loading model                 ...
Loading file names            ...
[WARNING] working with cap=2050000
File found:                  2050000
Filtering already processed: 2000000
Running over                  1206120 elements ...
0-1000
Dumping to disk ...
/home/ec2-user/SageMaker/mariano/repositories/refugee-research-problem-hrir/embeddings/item_representation_2.pickle
1000-2000
Dumping to disk ...
/home/ec2-user/SageMaker/mariano/repositories/refugee-research-problem-hrir/embeddings/item_representation_3.pickle
2000-3000


KeyboardInterrupt: 

# ONLY 50K

In [1]:
###########
# IMPORTS #
###########
from sentence_transformers import SentenceTransformer
import re
import pickle
import os

import sys
home_path=open('/home/ec2-user/SageMaker/mariano/repositories/refugee-research-problem-hrir/config/repository_path.txt').read()
sys.path.append(home_path)

from lib import tdmstudio
from lib import refugee_data_sources

####################
# INPUT AND OUTPUT #
####################
model_path='/home/ec2-user/SageMaker/mariano/sentence_bert/pretrained/all-MiniLM-L6-v2/'


output = os.path.join(home_path, f'embeddings/item_representation_50k_final.pickle')

print('Loading model                 ...')
model = SentenceTransformer(model_path)

print('Loading file names            ...')



unlabeled = refugee_data_sources.get_unlabeled_sample_files()
print(f'File found:                  {len(unlabeled)}')

vecs = model.encode(list(map(tdmstudio.get_title_and_text, map(tdmstudio.get_filename, unlabeled))))
item_representation =  {id_:vecs[ix,:] for ix,id_ in enumerate(unlabeled)}
print('Dumping to disk ...')
print(output)

with open(output,'wb') as writer:
    pickle.dump(item_representation, writer)

print('Done ! ')

Loading model                 ...
Loading file names            ...
File found:                  50000


NameError: name 'i' is not defined

Dumping to disk ...
/home/ec2-user/SageMaker/mariano/repositories/refugee-research-problem-hrir/embeddings/item_representation_50k_final.pickle
Done ! 
